In [41]:
import numpy as np
import pandas as pd
import os

In [42]:
from sklearn.model_selection import train_test_split
from shutil import copyfile

mainDataDir='/kaggle/input/multi-cancer/Multi Cancer/ALL'
benignDir=os.path.join(mainDataDir,'all_benign')
earlyDir=os.path.join(mainDataDir,'all_early')
preDir=os.path.join(mainDataDir,'all_pre')
proDir=os.path.join(mainDataDir,'all_pro')

In [43]:
trainDir = '/kaggle/working/training_data'
valDir = '/kaggle/working/validation_data'

In [44]:
os.makedirs(trainDir,exist_ok=True)
os.makedirs(valDir,exist_ok=True)

In [45]:
def split_and_copy(class_dir, train_output_dir, val_output_dir):
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith(('jpg', 'jpeg', 'png'))]

    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(train_output_dir, exist_ok=True)
    os.makedirs(val_output_dir, exist_ok=True)

    for img in train_images:
        copyfile(img, os.path.join(train_output_dir, os.path.basename(img)))

    for img in val_images:
        copyfile(img, os.path.join(val_output_dir, os.path.basename(img)))

split_and_copy(benignDir,os.path.join(trainDir,'all_benign'),os.path.join(valDir,'all_benign'))
split_and_copy(earlyDir,os.path.join(trainDir,'all_early'),os.path.join(valDir,'all_early'))
split_and_copy(preDir,os.path.join(trainDir,'all_pre'),os.path.join(valDir,'all_pre'))
split_and_copy(proDir,os.path.join(trainDir,'all_pro'),os.path.join(valDir,'all_pro'))

## Data Augmentation

In [46]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    '/kaggle/working/training_data',
    target_size=(224,224),
    class_mode='binary',
    batch_size=32
)

val_generator=val_datagen.flow_from_directory(
    '/kaggle/working/validation_data',
    target_size=(224,224),
    class_mode='binary',
    batch_size=32
)

Found 16000 images belonging to 4 classes.
Found 4000 images belonging to 4 classes.


## Training the Data

In [47]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Input,InputLayer,GlobalAveragePooling1D,BatchNormalization

In [48]:
numClasses=len(train_generator.class_indices)

In [49]:
imageShape=(224,224,3)

In [75]:
from tensorflow.keras.regularizers import l2

model = Sequential([
    Input(shape=imageShape),
    Conv2D(32,(3,3),activation='relu',kernel_regularizer=l2(0.001)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),
    Flatten(),
    Dense(128,activation='relu'),
    Dense(numClasses,activation='sigmoid')
])

In [77]:
from keras.optimizers import Adam

In [86]:
model.compile(
#     optimizer=Adam(learning_rate=0.001),
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [87]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,788,932 (10.64 MB)

 Trainable params: 2,788,932 (10.64 MB)

 Non-trainable params: 0 (0.00 B)

In [88]:
from keras.callbacks import EarlyStopping
earlystop= EarlyStopping(monitor='val_loss', patience=4,mode="max")

In [92]:
batch_size=16
steps_per_epoch = len(train_generator) // batch_size  
validation_steps = len(val_generator) // batch_size  
history = model.fit(train_generator,validation_data=val_generator,epochs=100,steps_per_epoch=steps_per_epoch,validation_steps=validation_steps,callbacks=[earlystop])

Epoch 1/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.7769 - loss: 0.4969 - val_accuracy: 0.7946 - val_loss: 0.4414
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.7979 - loss: 0.4818 - val_accuracy: 0.8795 - val_loss: 0.3537
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.8259 - loss: 0.4044 - val_accuracy: 0.8259 - val_loss: 0.4309
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.8004 - loss: 0.4750 - val_accuracy: 0.8884 - val_loss: 0.3350


In [93]:
# Evaluate on the training generator
train_results = model.evaluate(train_generator,batch_size=32)
print("Train loss, Train accuracy:", train_results)

# Evaluate on the validation generator
val_results = model.evaluate(val_generator)
print("Validation loss, Validation accuracy:", val_results)

500/500 ━━━━━━━━━━━━━━━━━━━━ 317s 634ms/step - accuracy: 0.8463 - loss: 0.3790
Train loss, Train accuracy: [0.3763342499732971, 0.8474375009536743]
125/125 ━━━━━━━━━━━━━━━━━━━━ 43s 347ms/step - accuracy: 0.8664 - loss: 0.3456
Validation loss, Validation accuracy: [0.35303589701652527, 0.8637499809265137]


In [94]:
def savingModel(model, model_name="model"): 
    model_filename = f"{model_name}.h5"
    model.save(model_filename)
    print(f"Model saved as {model_filename}")

In [95]:
savingModel(model,model_name="ALLCancer")

Model saved as ALLCancer.h5


In [96]:
from keras.models import load_model

In [97]:
def loadingModel(model_name="model"): 
    try:
        model = load_model(f"{model_name}.h5")
        model.summary()  # Print model summary if needed
        return model  # Return the loaded model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

In [98]:
loadedModel=loadingModel("ALLCancer")

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,788,934 (10.64 MB)

 Trainable params: 2,788,932 (10.64 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [99]:
def loadImg(imgPath): 
    img = tf.keras.preprocessing.image.load_img(imgPath, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array=img_array/255.0
    return img_array

In [100]:
import tensorflow as tf

In [101]:
def predictImagesInFolder(testFolder, class_name, model_name): 
    savedModel = loadingModel(model_name)
    print(savedModel)

    if savedModel is None:
        print("Model loading failed.")
        return

    for subdir, dirs, files in os.walk(testFolder): 
        if subdir == testFolder: 
            continue
        trueLabel = os.path.basename(subdir)
        for file in files: 
            img_path = os.path.join(subdir, file)
            img = loadImg(img_path)
            
            # Debugging prediction probabilities
            predictions = savedModel.predict(img)
#             print(f"Prediction Probabilities for {file}: {predictions}")

            predicted_class_index = np.argmax(predictions, axis=1)
            predicted_class = class_name[predicted_class_index[0]]
            print(f"Predicted class: {predicted_class} File: {file}")

In [102]:
def testFolder(folder): 
    testFolder=f"{folder}"
    return testFolder

In [103]:
ALLCancerTestFolder=testFolder("/kaggle/working/validation_data")

In [104]:
train_generator.class_indices

{'all_benign': 0, 'all_early': 1, 'all_pre': 2, 'all_pro': 3}

In [105]:
ALLCancerClassname={0:'all_benign', 1:'all_early', 2:'all_pre', 3:'all_pro'}


In [ ]:
predictImagesInFolder(ALLCancerTestFolder,ALLCancerClassname,model_name="ALLCancer")

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,788,934 (10.64 MB)

 Trainable params: 2,788,932 (10.64 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

<Sequential name=sequential_6, built=True>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Predicted class: all_pre File: all_pre_2618.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: all_pre File: all_pre_2689.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: all_pre File: all_pre_1990.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: all_pre File: all_pre_4439.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: all_pre File: all_pre_1216.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted class: all_pre File: all_pre_4936.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: all_pre File: all_pre_3702.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: all_pre File: all_pre_1702.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted class: all_pre File: all_pre_0415.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted class: all_pre File: all_pre_3281.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: all_pre File: all_pre_0881.jpg
1/1 ━━━━━━━